In [1]:
import numpy as np
import gmms

## Switch from Python to Cython backend

In [3]:
print('Distance tools implementation: {}'.format(gmms.distancetools.__implementation__))
print('CB10 implementation: {}'.format(gmms.CampbellBozorgnia2010.__implementation__))
print('CB11 implementation: {}'.format(gmms.CampbellBozorgnia2011.__implementation__))
print('CB19 implementation: {}'.format(gmms.CampbellBozorgnia2019.__implementation__))

Distance tools implementation: Python
CB10 implementation: Python
CB11 implementation: Python
CB19 implementation: Python


In [4]:
gmms.distancetools.select_backend('Cython')
gmms.CampbellBozorgnia2010.select_backend('Cython')
gmms.CampbellBozorgnia2019.select_backend('Cython')
gmms.CampbellBozorgnia2011.select_backend('Cython')

In [5]:
print('Distance tools implementation: {}'.format(gmms.distancetools.__implementation__))
print('CB10 implementation: {}'.format(gmms.CampbellBozorgnia2010.__implementation__))
print('CB11 implementation: {}'.format(gmms.CampbellBozorgnia2011.__implementation__))
print('CB19 implementation: {}'.format(gmms.CampbellBozorgnia2019.__implementation__))

Distance tools implementation: Cython
CB10 implementation: Cython
CB11 implementation: Cython
CB19 implementation: Cython


# Example: 1989 Loma Prieta Earthquake

## Earthquake parameters

In [8]:
M      = 6.93
hyp_Z  = 17.48
region = 'Global' # regionalized models for 'Japan', 'Italy', and 'China'

## Fault model

In [10]:
ULC_lat = 37.19
ULC_lon = -122.03
flength = 40.
fwidth  = 18
fstrike = 128.
fdip    = 70.
frake   = 140.
fZtor   = 3.85

## Site parameters

In [12]:
site_lat  = 37.825
site_lon  = -122.373
site_Vs30 = 155.11

## Convert to arrays

In [14]:
vals = []
for val in zip([ULC_lat,ULC_lon,flength,fwidth,fstrike,fdip,frake,fZtor,site_lat,site_lon,site_Vs30]):
    if isinstance(val[0],np.ndarray):
        vals.append(val[0])
    elif isinstance(val[0], (int, float)):
        vals.append(np.array([float(val[0])]))
    elif isinstance(val[0], (list, tuple)):
        vals.append(np.array(val[0], dtype=float))
    else:
        vals.append(val[0])
[ULC_lat,ULC_lon,flength,fwidth,fstrike,fdip,frake,fZtor,site_lat,site_lon,site_Vs30] = vals

## Compute fault parameters

In [16]:
[URC_lat,URC_lon] = gmms.faulttools.get_fault_URC(ULC_lat,ULC_lon,flength,fstrike)
[fnm,frv,fmech]   = gmms.faulttools.get_faulting_style(frake)

## Compute site parameters

In [18]:
site_Z1p0 = np.exp((-7.15/4)*np.log((site_Vs30**4+571**4)/(1360**4+571**4))) # BSSA14
site_Z2p5 = np.exp(7.089-1.144*np.log(site_Vs30))                            # CB13

In [19]:
[R_JB,R_rup,R_x] = gmms.distancetools.get_distances(site_lat,site_lon,ULC_lat,ULC_lon,URC_lat,URC_lon,fwidth,flength,fdip,fstrike,fZtor)

In [20]:
print('Rjb  = {:.2f} km'.format(R_JB[0]))
print('Rrup = {:.2f} km'.format(R_rup[0]))
print('Rx   = {:.2f} km'.format(R_x[0]))

Rjb  = 76.72 km
Rrup = 76.82 km
Rx   = -37.00 km


## Compute ground motion intensity measures

### CAVgm

In [23]:
[mu,phi,tau] = gmms.CampbellBozorgnia2010.CampbellBozorgnia2010(M,fdip[0],fZtor[0],R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv)
print('CAVstd = {:.2f} m/s'.format(np.exp(mu[0])))

CAVstd = 0.53 m/s


### CAVdp

In [25]:
flag_CAVstd = 1 # '0' for recorded, '1' for gmm-based
flag_PSV    = 0 # '0' for no PSV check, '1' otherwise

In [26]:
[mu,phi,tau] = gmms.CampbellBozorgnia2011.CampbellBozorgnia2011(M,fdip[0],fZtor[0],R_JB,R_rup,R_x,site_Vs30,site_Z1p0,site_Z2p5,fnm,frv,flag_CAVstd,flag_PSV)

In [27]:
print('CAVdp = {:.2f} m/s'.format(np.exp(mu[0])))

CAVdp = 0.37 m/s


### CAV

In [29]:
[mu,phi,tau] = gmms.CampbellBozorgnia2019.CampbellBozorgnia2019(1,M,fwidth[0],fdip[0],fZtor[0],hyp_Z,R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv,region)
print('CAV = {:.2f} m/s'.format(np.exp(mu[0])))

CAV = 5.53 m/s


### Ia

In [31]:
[mu,phi,tau] = gmms.CampbellBozorgnia2019.CampbellBozorgnia2019(0,M,fwidth[0],fdip[0],fZtor[0],hyp_Z,R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv,region)

In [32]:
print('Ia = {:.2f} m/s'.format(np.exp(mu[0])))

Ia = 0.20 m/s
